In [1]:
import pandas as pd
import re

## Import data and extract (E)

In [2]:
df = pd.read_csv('data/brazil_example.csv')
df

,time,country,vehicle,metric_type,device,target_audience,metric_numbers,cost_usd
0,13/01/2018,brazil,facebook,impressions,mobile,19-49,55150.0,188.730000
1,13/01/2018,brazil,facebook,impressions,mobile,19-49,1260949.0,2762.380000
2,13/01/2018,brazil,instagram,impressions,mobile,19-49,55524.0,117.430000
3,13/01/2018,brazil,instagram,impressions,mobile,19-49,276979.0,2037.620000
4,20/01/2018,brazil,facebook,impressions,mobile,19-49,44897.0,122.550000
...,...,...,...,...,...,...,...,...
60845,29/06/2024,brazil,instagram,impressions,mobile_app,not_available,73856.0,138.433053
60846,29/06/2024,brazil,instagram,impressions,mobile_app,not_available,13834.0,23.116623
60847,29/06/2024,brazil,instagram,impressions,mobile_app,not_available,154523.0,297.399275
60848,29/06/2024,brazil,instagram,impressions,mobile_app,not_available,37229.0,467.654870


In [3]:
df['vehicle'].unique()

array(['facebook', 'instagram', 'ooh', 'tv', 'youtube', 'digitalvideo',
       'digitaldisplayandsearch', 'cinema', 'twitter', 'print',
       'programmaticvideo', 'programmaticdisplay', 'influencer',
       'partnership', 'netflix', 'dooh', 'otherdigital', 'tvsub'],
      dtype=object)

In [4]:
df['metric_type'].unique()

array(['impressions', 'grp', 'spend', 'followers', 'net grp', 'views',
       'ratings', 'view', 'engagement', 'insertion'], dtype=object)

In [5]:
df['device'].unique()

array(['mobile', 'no_device', 'desktop', 'multiscreen', 'not_available',
       'mobile app', 'mobile web', 'tv', 'dooh', 'cross device',
       'mobile_app', 'mobile_web', 'desktop_feed', 'desktop_video'],
      dtype=object)

In [6]:
df['target_audience'].unique()

array(['19-49', 'not_available', '19-39', '19-34', '19-44', '20-39',
       '19+', '21-39', '19-25', '21+', '19-65', '21-34', '18-44', '20-49',
       '25-49', '25-44', '30-49', '21-49', '18-34', '35-44', '18-54',
       '18+', '19-59', '21-54', '19-54', '19-29', '-',
       '19-59 & behavioral/interest', '19-39 & behavioral/interest',
       '21-100', '18-54 & behavioral/interest', '20-59',
       '20-59 & behavioral/interest', '19-59 & retargeting',
       '20-59 & retargeting', '21-54 & behavioral/interest',
       'adults 19-39', 'adults 40-59', 'adults 19-59',
       'ages 25-44 drinkers', 'ages 18-44 drinkers',
       'ages 18-34 drinkers', 'age 18+ drinkers', 'adults 19-49',
       'adults 40-49'], dtype=object)

## Tratament of datas and transformation (T)

In [7]:
df_target = df.copy()

def transform_XX_XX_to_18(age_range):
    """
    +18 : all drinkers +18
    not_availble : not possible available
    retargeting : behavioral/interest retargeting
    """
    # "XX-XX"
    if re.match(r'^\d{2}-\d{2,3}$', age_range):
        return "+18"  
    #"XX-XX" em "+18"
    if re.match(r'^\d{2}\+$', age_range):
        return "+18"
    # "adults XX-XX", "ages XX-XX drinkers"
    if re.search(r'(adults|ages)\s\d{2}-\d{2}', age_range, re.IGNORECASE):
        return "+18"
    if age_range == 'age 18+ drinkers':
        return "+18"
    if age_range == '-' :
        return "not_available"
    if "behavioral/interest" in age_range or "retargeting" in age_range:
        return "retargeting"

    return age_range


df_target['target_audience'] = df_target['target_audience'].apply(transform_XX_XX_to_18)


df_target['target_audience'].unique()

array(['+18', 'not_available', 'retargeting'], dtype=object)

In [8]:
df_target_device = df_target.copy()

def transform_device(device):
    """
    mobile : all mobile variants
    desktop : all desktop variants 
    web : all web variants
    not available : all no available variants
    """
    mobile_variants = ["mobile", "mobile_app", "mobile app"]
    desktop_variants = ["desktop", "desktop_video", "desktop_feed"]
    web_variants = ["mobile_web", "mobile web"]
    no_available_variants = ["no_device", "not_available"]
    
    if device in mobile_variants:
        return "mobile app"
    if device in desktop_variants:
        return "desktop"
    if device in web_variants:
        return "web"
    if device in no_available_variants:
        return "not available"
    return device

df_target_device["device"] = df_target_device["device"].apply(transform_device)
df_target_device["device"].unique()

array(['mobile app', 'not available', 'desktop', 'multiscreen', 'web',
       'tv', 'dooh', 'cross device'], dtype=object)

In [9]:
df_target_device_metric = df_target_device.copy()

def transform_metric(metric):
    """
    viewws : all views variants
    """
    views_variants = ["views", "view"]
    if metric in views_variants:
        return "view"
    return metric

df_target_device_metric["metric_type"] = df_target_device_metric["metric_type"].apply(transform_metric)
df_target_device_metric["metric_type"].unique()

array(['impressions', 'grp', 'spend', 'followers', 'net grp', 'view',
       'ratings', 'engagement', 'insertion'], dtype=object)

## Load of data to file .csv

In [11]:
df_target_device_metric.to_csv("data_result.csv")